In [1]:
import pandas as pd
import numpy as np

# 明确所有列名（请按你的表头实际顺序核对）
columns = [
    'DateTime', 'Global_active_power', 'Global_reactive_power', 'Voltage', 'Global_intensity',
    'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3',
    'RR', 'NBJRR1', 'NBJRR5', 'NBJRR10', 'NBJBROU'
]
f='/aistor/aispeech/hpc_stor01/home/xuruoxi00sx/ttt/test.csv'
# 读取数据
df = pd.read_csv(
    f,
    parse_dates=['DateTime'],
    na_values=['?', 'NA', 'null', '', ' ', 'NaN']
)

# 确认所有数值列都为float型
num_cols = [col for col in columns if col != 'DateTime']
for col in num_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# 删除包含缺失值的行
df.dropna(inplace=True)

# 添加日期列
df['Date'] = df['DateTime'].dt.date

# 异常值处理（IQR法）
def replace_outliers_with_day_mean(x):
    q1 = x.quantile(0.25)
    q3 = x.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    mask = (x < lower) | (x > upper)
    mean_wo = x[~mask].mean()
    x[mask] = mean_wo
    return x

for col in num_cols:
    df[col] = df.groupby('Date')[col].transform(replace_outliers_with_day_mean)

# 计算 sub_metering_remainder
df['sub_metering_remainder'] = (
    df['Global_active_power'] * 1000 / 60
    - (df['Sub_metering_1'] + df['Sub_metering_2'] + df['Sub_metering_3'])
)

# 日聚合
agg_dict = {
    'Global_active_power': 'sum',
    'Global_reactive_power': 'sum',
    'Sub_metering_1': 'sum',
    'Sub_metering_2': 'sum',
    'Sub_metering_3': 'sum',
    'sub_metering_remainder': 'sum',
    'Voltage': 'mean',
    'Global_intensity': 'mean',
    'RR': 'first',
    'NBJRR1': 'first',
    'NBJRR5': 'first',
    'NBJRR10': 'first',
    'NBJBROU': 'first'
}

daily = df.groupby('Date').agg(agg_dict).reset_index()

# 保存结果
daily.to_csv('daily_test.csv', index=False)

In [3]:
import pandas as pd

# 读取数据
test_data = pd.read_csv('/aistor/aispeech/hpc_stor01/home/xuruoxi00sx/ttt/daily_test.csv')

# 判断是否有任何 NaN
has_nan = test_data.isnull().values.any()
print(f"是否有 NaN 值: {has_nan}")

# 获取每列的缺失值数量
nan_per_column = test_data.isnull().sum()

# 找出存在缺失值的列名
columns_with_nan = nan_per_column[nan_per_column > 0].index.tolist()

# 输出存在缺失值的列名
print("存在缺失值的列名：")
print(columns_with_nan)

# 找出存在缺失值的行号
rows_with_nan = test_data[test_data.isnull().any(axis=1)].index.tolist()

# 输出存在缺失值的行号
print("存在缺失值的行号：")
print(rows_with_nan)


是否有 NaN 值: False
存在缺失值的列名：
[]
存在缺失值的行号：
[]
